<a href="https://colab.research.google.com/github/DarshitSarda/AlgoStrategies/blob/main/Bollinger_Doji_Engulfing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Doji Vs Engulfing Patterns

First we check the bollinger bands on the chart. We wait for the price-action to go above or below the defined bollinger band levels, then finally we wait for a buying signal in the form of a bullish doji/bullish engulfing or vice-versa in the case of shorting/selling

In [11]:
import yfinance as yf
import pandas as pd

dataF = yf.download("EURUSD=X", start="2023-5-10", end="2024-5-10", interval='1h')
#df.iloc[:,:]

[*********************100%%**********************]  1 of 1 completed


In [1]:
!pip install pandas-ta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas-ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218907 sha256=ace70451604512ff51ab2b4b2f2d7435bad53546816c429c6a96de3f64cc12b4
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas-ta


In [12]:
import pandas_ta as ta
my_bbands = ta.bbands(dataF.Close, length=30, std=1.5)
dataF=dataF.join(my_bbands)

In [13]:
dataF.columns

Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'BBL_30_1.5',
       'BBM_30_1.5', 'BBU_30_1.5', 'BBB_30_1.5', 'BBP_30_1.5'],
      dtype='object')

Here, I changed the code a bit so that instead of checking for buy/sell signals outside the bollinger bands, we consider it inside also. Checking around the BBM (Bollinger Band Midline)
For buy: above BBL and below BBM
For sell: above BBM and below BBH

In [31]:
def bollinger_doji_signal(df):
    #bullish signal
    if ( #df.Close.iloc[-1] < df['BBL_30_1.5'].iloc[-1] and
       df.Close.iloc[-1] > df['BBL_30_1.5'].iloc[-1] and
       df.Close.iloc[-1] < df['BBM_30_1.5'].iloc[-1] and
       df.Close.iloc[-1] > df.Open.iloc[-1] and
       df.Close.iloc[-2] == df.Open.iloc[-2] and
       df.Close.iloc[-3] < df.Open.iloc[-3] ):
        return 2

    #bearish signal
    elif (#df.Close.iloc[-1] > df['BBU_30_1.5'].iloc[-1] and
       df.Close.iloc[-1] < df['BBU_30_1.5'].iloc[-1] and
       df.Close.iloc[-1] > df['BBM_30_1.5'].iloc[-1] and
       df.Close.iloc[-1] < df.Open.iloc[-1] and
       df.Close.iloc[-2] == df.Open.iloc[-2] and
       df.Close.iloc[-3] > df.Open.iloc[-3] ):
         return 1

    #nosignal
    else:
        return 0

signal = [0]*len(dataF)
for i in range(20,len(dataF)):
    df = dataF[i-3:i+1]
    signal[i]= bollinger_doji_signal(df)
dataF["bollinger_doji_signal"] = signal

In [32]:
import numpy as np
def pointpos(x):
    if x['bollinger_doji_signal']==1:
        return x['High']+0.5e-3
    elif x['bollinger_doji_signal']==2:
        return x['Low']-0.5e-3
    else:
        return np.nan
dataF['pointpos'] = dataF.apply(lambda row: pointpos(row), axis=1)

In [33]:
dataF.reset_index(inplace=True)
#dataF[dataF["bollinger_doji_signal"]!=0]

In [34]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime

st = 700
dfpl = dataF[st:st+250].copy()
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close']),
                go.Scatter(x=dfpl.index, y=dfpl['BBL_30_1.5'], line=dict(color='blue', width=1), name="BBL"),
                go.Scatter(x=dfpl.index, y=dfpl['BBU_30_1.5'], line=dict(color='blue', width=1), name="BBU")])

fig.add_scatter(x=dfpl.index, y=dfpl['pointpos'], mode="markers",
                marker=dict(size=5, color="MediumPurple"),
                name="Signal")
fig.update_layout(autosize=False, width=1000, height=600)
fig.show()

In [35]:
def SIGNAL():
    return dataF.bollinger_doji_signal

In [36]:
!pip install backtesting

In [37]:
from backtesting import Strategy
from backtesting import Backtest
import backtesting

class BreakOut(Strategy):
    initsize = 0.5
    mysize = initsize
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)

    def next(self):
        super().next()
        TPSLRatio = 1.5

        if self.signal1==2 and len(self.trades)==0:
            sl1 = min(self.data.Low[-2], self.data.Low[-1])
            tp1 = self.data.Close[-1] + abs(self.data.Close[-1]-sl1)*TPSLRatio
            self.buy(sl=sl1, tp=tp1, size=self.mysize)

        elif self.signal1==1 and len(self.trades)==0:
            sl1 = max(self.data.High[-2], self.data.High[-1])
            tp1 = self.data.Close[-1] - abs(sl1-self.data.Close[-1])*TPSLRatio
            self.sell(sl=sl1, tp=tp1, size=self.mysize)

bt = Backtest(dataF, BreakOut, cash=1000, margin=1/10)
stat = bt.run()
stat

<ipython-input-37-75d0dba8e072>:26: UserWarning:

Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.



Start                                     0.0
End                                    6204.0
Duration                               6204.0
Exposure Time [%]                    4.899275
Equity Final [$]                  1045.023088
Equity Peak [$]                   1045.023088
Return [%]                           4.502309
Buy & Hold Return [%]               -1.650483
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                   -3.626494
Avg. Drawdown [%]                   -0.701084
Max. Drawdown Duration                 4322.0
Avg. Drawdown Duration             424.333333
# Trades                                 76.0
Win Rate [%]                        43.421053
Best Trade [%]                       0.339617
Worst Trade [%]                     -0.244323
Avg. Trade [%]                    

In [23]:
bt.plot()

/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning:

found multiple competing values for 'toolbar.active_drag' property; using the latest value

/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning:

found multiple competing values for 'toolbar.active_scroll' property; using the latest value



GridPlot(id='p1327', ...)

In [38]:
def bollinger_engulfing_signal(df):
    #bullish signal
    if (#df.Close.iloc[-1] < df['BBL_30_1.5'].iloc[-1] and
       df.Close.iloc[-1] > df['BBL_30_1.5'].iloc[-1] and
       df.Close.iloc[-1] < df['BBM_30_1.5'].iloc[-1] and
       df.Close.iloc[-1] > df.Open.iloc[-1] and
       df.Close.iloc[-2] < df.Open.iloc[-2] and
       df.Open.iloc[-1] < df.Close.iloc[-2] and
       df.Close.iloc[-1] > df.Open.iloc[-2] ):
        return 2

    #bearish signal
    elif (#df.Close.iloc[-1] > df['BBU_30_1.5'].iloc[-1] and
       df.Close.iloc[-1] < df['BBU_30_1.5'].iloc[-1] and
       df.Close.iloc[-1] > df['BBM_30_1.5'].iloc[-1] and
       df.Close.iloc[-1] < df.Open.iloc[-1] and
       df.Close.iloc[-2] > df.Open.iloc[-2] and
       df.Open.iloc[-1] > df.Close.iloc[-2] and
       df.Close.iloc[-1] < df.Open.iloc[-2] ):
         return 1

    #nosignal
    else:
        return 0

signal = [0]*len(dataF)
for i in range(20,len(dataF)):
    df = dataF[i-3:i+1]
    signal[i]= bollinger_engulfing_signal(df)
dataF["bollinger_engulfing_signal"] = signal

In [39]:
def pointpos(x):
    if x['bollinger_engulfing_signal']==1:
        return x['High']+1e-3
    elif x['bollinger_engulfing_signal']==2:
        return x['Low']-1e-3
    else:
        return np.nan
dataF['pointpos'] = dataF.apply(lambda row: pointpos(row), axis=1)

In [40]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime

st = 200
dfpl = dataF[st:st+250].copy()
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close']),
                go.Scatter(x=dfpl.index, y=dfpl['BBL_30_1.5'], line=dict(color='blue', width=1), name="BBL"),
                go.Scatter(x=dfpl.index, y=dfpl['BBU_30_1.5'], line=dict(color='blue', width=1), name="BBU")])

fig.add_scatter(x=dfpl.index, y=dfpl['pointpos'], mode="markers",
                marker=dict(size=5, color="MediumPurple"),
                name="Signal")
fig.update_layout(autosize=False, width=1000, height=600)
fig.show()

In [45]:
def SIGNAL():
    return dataF.bollinger_engulfing_signal

In [47]:
class BreakOut(Strategy):
    initsize = 0.5
    mysize = initsize
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)

    def next(self):
        super().next()
        TPSLRatio = 1.5

        if self.signal1==2 and len(self.trades)==0:
            sl1 = self.data.Low[-2]
            tp1 = self.data.Close[-1] + abs(self.data.Close[-1]-sl1)*TPSLRatio
            self.buy(sl=sl1, tp=tp1, size=self.mysize)

        elif self.signal1==1 and len(self.trades)==0:
            sl1 = self.data.High[-2]
            tp1 = self.data.Close[-1] - abs(sl1-self.data.Close[-1])*TPSLRatio
            self.sell(sl=sl1, tp=tp1, size=self.mysize)

bt = Backtest(dataF, BreakOut, cash=1000, margin=1/10)
stat = bt.run()
stat

<ipython-input-47-2ab5e950ccdd>:22: UserWarning:

Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.



Start                                     0.0
End                                    6204.0
Duration                               6204.0
Exposure Time [%]                    9.830782
Equity Final [$]                  1090.742069
Equity Peak [$]                   1113.266035
Return [%]                           9.074207
Buy & Hold Return [%]               -1.650483
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                   -8.304688
Avg. Drawdown [%]                   -1.125535
Max. Drawdown Duration                 1911.0
Avg. Drawdown Duration                  236.0
# Trades                                 73.0
Win Rate [%]                        47.945205
Best Trade [%]                       0.544371
Worst Trade [%]                     -0.379323
Avg. Trade [%]                    

In [43]:
bt.plot()

/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning:

found multiple competing values for 'toolbar.active_drag' property; using the latest value

/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning:

found multiple competing values for 'toolbar.active_scroll' property; using the latest value



GridPlot(id='p2027', ...)